In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
from google.cloud import bigquery
from google.cloud.bigquery import magics
magics.context.project = "grand-icon-475820-e5"


In [3]:
%%bigquery sales_by_weekday
WITH event_summary AS (
  SELECT
    f.event_date,
    d.weekday,
    d.weekday_name,
    SUM(f.total_spend) AS total_spend_in_event,
    SUM(f.num_tickets) AS total_tickets_in_event,
    COUNT(DISTINCT f.customer_id) AS customers_in_event
  FROM `grand-icon-475820-e5.pwhl_analytics_assessment.fact_ticket_sales` AS f
  JOIN `grand-icon-475820-e5.pwhl_analytics_assessment.dim_date` AS d
    ON f.date_id = d.date_id
  GROUP BY f.event_date, d.weekday, d.weekday_name
)
SELECT
  weekday,
  weekday_name,
  ROUND(AVG(total_spend_in_event), 2) AS avg_total_sales_per_event,
  ROUND(AVG(total_spend_in_event / customers_in_event), 2) AS avg_spend_per_customer,
  ROUND(AVG(total_tickets_in_event), 2) AS avg_tickets_sold_per_event,
  ROUND(AVG(total_tickets_in_event / customers_in_event), 2) AS avg_tickets_sold_per_customer,
  SUM(total_tickets_in_event) AS total_tickets_all_events
FROM event_summary
GROUP BY weekday, weekday_name
ORDER BY weekday;




Query is running:   0%|          |

Downloading:   0%|          |

In [4]:
sales_by_weekday.head(10)

,weekday,weekday_name,avg_total_sales_per_event,avg_spend_per_customer,avg_tickets_sold_per_event,avg_tickets_sold_per_customer,total_tickets_all_events
0,1,monday,359933.75,259.44,4879.75,3.52,19519
1,2,tuesday,297690.00,264.61,3900.00,3.47,3900
2,3,wednesday,372923.33,264.61,4934.00,3.51,14802
3,4,thursday,566550.00,267.87,7439.00,3.52,7439
4,5,friday,364758.75,271.09,4733.25,3.53,18933
5,6,saturday,257885.00,269.26,3361.00,3.51,6722


In [5]:
%%bigquery sales_by_temp
WITH event_summary AS (
  SELECT
    f.event_date,
    w.avg_temp_c,
    CASE
      WHEN w.avg_temp_c < 0 THEN 'Below 0°C'
      ELSE 'Above or equal 0°C'
    END AS temp_category,
    SUM(f.total_spend) AS total_spend_in_event,
    SUM(f.num_tickets) AS total_tickets_in_event,
    COUNT(DISTINCT f.customer_id) AS customers_in_event
  FROM `grand-icon-475820-e5.pwhl_analytics_assessment.fact_ticket_sales` AS f
  JOIN `grand-icon-475820-e5.pwhl_analytics_assessment.dim_weather` AS w
    ON f.weather_id = w.weather_id
  GROUP BY f.event_date, w.avg_temp_c, temp_category
)

SELECT
  temp_category,
  ROUND(AVG(total_spend_in_event), 2) AS avg_total_sales_per_event,
  ROUND(AVG(total_spend_in_event / customers_in_event), 2) AS avg_spend_per_customer,
  ROUND(AVG(total_tickets_in_event), 2) AS avg_tickets_sold_per_event,
  ROUND(AVG(total_tickets_in_event / customers_in_event), 2) AS avg_tickets_sold_per_customer,
  SUM(total_tickets_in_event) AS total_tickets_all_events
FROM event_summary
GROUP BY temp_category
ORDER BY temp_category;


Query is running:   0%|          |

Downloading:   0%|          |

In [6]:
sales_by_temp.head()

,temp_category,avg_total_sales_per_event,avg_spend_per_customer,avg_tickets_sold_per_event,avg_tickets_sold_per_customer,total_tickets_all_events
0,Above or equal 0°C,430546.0,269.64,5629.4,3.53,28147
1,Below 0°C,324482.0,263.88,4316.8,3.51,43168


In [7]:
%%bigquery section_capacity_sales
SELECT
  f.event_date,
  v.home_city,
  v.section,
  v.section_capacity,
  SUM(f.num_tickets) AS total_tickets_sold,
  ROUND(SUM(f.num_tickets) / v.section_capacity * 100, 2) AS percent_full
FROM `grand-icon-475820-e5.pwhl_analytics_assessment.fact_ticket_sales` AS f
JOIN `grand-icon-475820-e5.pwhl_analytics_assessment.dim_venue` AS v
  ON f.venue_id = v.venue_id
GROUP BY
  f.event_date,
  v.home_city,
  v.section,
  v.section_capacity
ORDER BY
  v.home_city,
  v.section,
  f.event_date,
  percent_full DESC;


Query is running:   0%|          |

Downloading:   0%|          |

In [8]:
section_capacity_sales.head(75)

,event_date,home_city,section,section_capacity,total_tickets_sold,percent_full
0,2025-01-04,boston,club,1147,552,48.13
1,2025-01-31,boston,club,1134,420,37.04
2,2025-01-04,boston,lower bowl,2677,1185,44.27
3,2025-01-31,boston,lower bowl,2646,1082,40.89
4,2025-01-04,boston,standing room,382,135,35.34
...,...,...,...,...,...,...
70,2025-02-24,vancouver,club,1219,696,57.10
71,2025-02-24,vancouver,lower bowl,2844,1768,62.17
72,2025-02-24,vancouver,standing room,406,329,81.03
73,2025-02-24,vancouver,suite,812,541,66.63


In [9]:
%%bigquery sales_performance_by_city
WITH event_summary AS (
  SELECT
    f.event_date,
    v.home_city,
    SUM(f.total_spend) AS total_spend_in_event,
    SUM(f.num_tickets) AS total_tickets_in_event,
    COUNT(DISTINCT f.customer_id) AS customers_in_event
  FROM `grand-icon-475820-e5.pwhl_analytics_assessment.fact_ticket_sales` AS f
  JOIN `grand-icon-475820-e5.pwhl_analytics_assessment.dim_venue` AS v
    ON f.venue_id = v.venue_id
  GROUP BY f.event_date, v.home_city
)
SELECT
  home_city,
  ROUND(AVG(total_spend_in_event), 2) AS avg_total_sales_per_event,
  ROUND(AVG(total_spend_in_event / customers_in_event), 2) AS avg_spend_per_customer,
  ROUND(AVG(total_tickets_in_event), 2) AS avg_tickets_sold_per_event,
  ROUND(AVG(total_tickets_in_event / customers_in_event), 2) AS avg_tickets_sold_per_customer,
  ROUND(AVG(total_spend_in_event / total_tickets_in_event), 2) AS avg_ticket_price,
  SUM(total_tickets_in_event) AS total_tickets_all_events
FROM event_summary
GROUP BY home_city
ORDER BY avg_total_sales_per_event DESC;


Query is running:   0%|          |

Downloading:   0%|          |

In [10]:
sales_performance_by_city.head(10)

,home_city,avg_total_sales_per_event,avg_spend_per_customer,avg_tickets_sold_per_event,avg_tickets_sold_per_customer,avg_ticket_price,total_tickets_all_events
0,seattle,479977.5,273.48,6180.0,3.52,77.68,12360
1,new york,409767.5,266.67,5393.0,3.51,75.87,10786
2,vancouver,380130.0,261.44,5122.0,3.52,74.22,5122
3,toronto,375127.5,265.87,4997.0,3.54,75.05,9994
4,minnesota,350175.0,259.23,4682.5,3.48,74.51,9365
5,ottawa,332195.0,260.48,4457.5,3.48,74.77,8915
6,montreal,301555.0,264.07,4012.5,3.51,75.19,8025
7,boston,259912.5,272.97,3374.0,3.55,77.00,6748


In [11]:
%%bigquery sales_channels_by_city
WITH city_channel_sales AS (
  SELECT
    v.home_city,
    c.purchase_channel,
    SUM(f.num_tickets) AS total_tickets_sold
  FROM `grand-icon-475820-e5.pwhl_analytics_assessment.fact_ticket_sales` AS f
  JOIN `grand-icon-475820-e5.pwhl_analytics_assessment.dim_channel` AS c
    ON f.channel_id = c.channel_id
  JOIN `grand-icon-475820-e5.pwhl_analytics_assessment.dim_venue` AS v
    ON f.venue_id = v.venue_id
  GROUP BY v.home_city, c.purchase_channel
)

SELECT
  home_city,
  purchase_channel,
  total_tickets_sold,
  ROUND(
    total_tickets_sold / SUM(total_tickets_sold) OVER (PARTITION BY home_city) * 100,
    2
  ) AS percent_of_city_tickets
FROM city_channel_sales
ORDER BY home_city, percent_of_city_tickets DESC;


Query is running:   0%|          |

Downloading:   0%|          |

In [12]:
sales_channels_by_city.head(50)

,home_city,purchase_channel,total_tickets_sold,percent_of_city_tickets
0,boston,online,1381,20.47
1,boston,group sales,1351,20.02
2,boston,box office,1343,19.90
3,boston,mobile app,1340,19.86
4,boston,season ticket,1333,19.75
5,minnesota,mobile app,2011,21.47
6,minnesota,group sales,1873,20.00
7,minnesota,online,1856,19.82
8,minnesota,season ticket,1840,19.65
9,minnesota,box office,1785,19.06


In [13]:
%%bigquery repeat_customers_by_city
WITH customer_city_sales AS (
  SELECT
    v.home_city,
    c.acct_id,
    COUNT(DISTINCT f.event_date) AS event_count
  FROM `grand-icon-475820-e5.pwhl_analytics_assessment.fact_ticket_sales` AS f
  JOIN `grand-icon-475820-e5.pwhl_analytics_assessment.dim_venue` AS v
    ON f.venue_id = v.venue_id
  JOIN `grand-icon-475820-e5.pwhl_analytics_assessment.dim_customer` AS c
    ON f.customer_id = c.customer_id
  GROUP BY v.home_city, c.acct_id
)
SELECT
  home_city,
  COUNTIF(event_count > 1) AS repeat_customers,
  COUNT(*) AS total_customers,
  ROUND(COUNTIF(event_count > 1) / COUNT(*) * 100, 2) AS repeat_customer_pct
FROM customer_city_sales
GROUP BY home_city
ORDER BY repeat_customer_pct DESC;

Query is running:   0%|          |

Downloading:   0%|          |

In [14]:
repeat_customers_by_city.head(10)

,home_city,repeat_customers,total_customers,repeat_customer_pct
0,minnesota,0,2693,0.0
1,boston,0,1904,0.0
2,toronto,0,2825,0.0
3,ottawa,0,2562,0.0
4,new york,0,3068,0.0
5,montreal,0,2284,0.0
6,seattle,0,3514,0.0
7,vancouver,0,1454,0.0


In [15]:
%%bigquery attendance_per_game
WITH per_section AS (
  SELECT
    v.event_date,
    v.home_city,
    v.section,
    MAX(v.section_capacity) AS section_capacity,
    SUM(f.num_tickets)       AS tickets_sold
  FROM `grand-icon-475820-e5.pwhl_analytics_assessment.fact_ticket_sales` AS f
  JOIN `grand-icon-475820-e5.pwhl_analytics_assessment.dim_venue` AS v
    ON f.venue_id = v.venue_id
  GROUP BY v.event_date, v.home_city, v.section
)
SELECT
  event_date,
  home_city,
  SUM(section_capacity) AS total_capacity,
  SUM(tickets_sold)     AS total_attendance,
  ROUND(100 * SUM(tickets_sold) / SUM(section_capacity), 2) AS pct_attendance
FROM per_section
GROUP BY event_date, home_city
ORDER BY event_date, home_city;

Query is running:   0%|          |

Downloading:   0%|          |

In [16]:
attendance_per_game.head(20)

,event_date,home_city,total_capacity,total_attendance,pct_attendance
0,2025-01-01,minnesota,7773,5157,66.35
1,2025-01-04,boston,7648,3522,46.05
2,2025-01-08,toronto,8155,5520,67.69
3,2025-01-13,ottawa,8468,5715,67.49
4,2025-01-17,new york,8460,3347,39.56
5,2025-01-22,montreal,7520,4125,54.85
6,2025-01-27,minnesota,8020,4208,52.47
7,2025-01-31,boston,7560,3226,42.67
8,2025-02-03,toronto,7848,4474,57.01
9,2025-02-08,ottawa,7730,3200,41.40
